In [10]:
%matplotlib inline
from IPython.core.pylabtools import figsize

from functools import partial
import itertools
import re

import matplotlib.gridspec as gridspec
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pylab as pl
import scipy as sp
from graphviz import Digraph
import pydot
from IPython.display import Image
import pandas as pd
# import seaborn as sns

print(__doc__)
import pandas as pd
from patsy import dmatrices
import scipy as sc
import scipy.optimize as opt
import statsmodels.api as sm

import sklearn as sk


from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import TransformerMixin
from sklearn.pipeline import BaseEstimator
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.utils.metaestimators import if_delegate_has_method
from sklearn.utils import column_or_1d
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union


from sklearn.datasets import samples_generator
from sklearn import datasets

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import sklearn.feature_extraction as fe

import sklearn.feature_selection as fs
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import VarianceThreshold

import sklearn.linear_model as lm
import sklearn.grid_search as gs
from sklearn.svm import SVC
from sklearn.decomposition import PCA


import sklearn.preprocessing as preprocessing
import sklearn.cross_validation as cv
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc

from sklearn_pandas import DataFrameMapper, cross_val_score

import matplotlib.pyplot as plt
import patsy
from statsmodels.sandbox import cox

# from patsylearn import *

import joblib as jb


from datasu.auc import *
from datasu.dicts import *
from datasu.files import *
from datasu.pandas import *
from datasu.persist import *


import pandas as pd
!which python


Automatically created module for IPython interactive environment
/home/sasha/.pyenv/versions/anaconda2-4.2.0/bin/python


### input data

In [118]:
cust_jrns_int = [
    {'customer_id':1, 'int_id': 1, 'jrn_id': 1, 'topics':'a', 'channel': 'web', 'step_num':0 },
    {'customer_id':1, 'int_id': 2, 'jrn_id': 1, 'topics':'b', 'channel': 'phone','step_num':1 },
    {'customer_id':1, 'int_id': 3, 'jrn_id': 1, 'topics':'a', 'channel': 'phone', 'step_num':2 },
    {'customer_id':1, 'int_id': 4, 'jrn_id': 2, 'topics':'b', 'channel': 'web', 'step_num':0 },
    {'customer_id':1, 'int_id': 5, 'jrn_id': 2, 'topics':'a', 'channel': 'phone','step_num':1 },    
]
df_cust_jrns_int = pd.DataFrame(cust_jrns_int, columns=['customer_id', 'jrn_id','int_id', 'channel', 'topics', 'step_num'])
# df_cust_jrns_int.set_index(['customer_id','jrn_id', 'int_id'],inplace=True)
df_cust_jrns_int

,customer_id,jrn_id,int_id,channel,topics,step_num
0,1,1,1,web,a,0
1,1,1,2,phone,b,1
2,1,1,3,phone,a,2
3,1,2,4,web,b,0
4,1,2,5,phone,a,1


In [119]:
dg1 = df_cust_jrns_int.groupby(by=['customer_id','jrn_id'], axis=0) 

def numerize(df):
    jrn_len = len(df)    
    df.sort_values(by=['int_id'],inplace=True)
    df['step_num'] = range(jrn_len)
    df['is_last_step'] = (jrn_len-1)*[False] + [True]
    return df

dg1.apply(numerize)

,customer_id,jrn_id,int_id,channel,topics,step_num,is_last_step
0,1,1,1,web,a,0,False
1,1,1,2,phone,b,1,False
2,1,1,3,phone,a,2,True
3,1,2,4,web,b,0,False
4,1,2,5,phone,a,1,True


### prepare data

In [120]:
grpby_columns = ['customer_id','jrn_id']

concat = lambda arr: ",".join(arr)
totuple = lambda arr: tuple(map(lambda csv: csv.split(','), arr))
def to_markov1_features(lists):
    import itertools
    from operator import concat    
    m1_pairs = [map(lambda l: "_".join(l), itertools.product(f_t[0],f_t[1])) for f_t in zip(lists[:-1], lists[1:])]
    return reduce(concat, m1_pairs)


aggs = build_df_agg(grpby_columns=['jrn'], agg_columns=['topics','channel'],
                    agg_funcs={'m0':concat, 'm1':lambda v: concat(to_markov1_features(totuple(v)))})

gr_cust_jrns = df_cust_jrns_int.groupby(by=grpby_columns, axis=0) 
df_jrns = gr_cust_jrns.agg(aggs)
df_jrns.columns = flatten_columns(df_jrns)
df_jrns = df_jrns.reset_index()
df_jrns

,customer_id,jrn_id,topics_jrn_m0,topics_jrn_m1,channel_jrn_m0,channel_jrn_m1
0,1,1,"a,b,a","a_b,b_a","web,phone,phone","web_phone,phone_phone"
1,1,2,"b,a",b_a,"web,phone",web_phone


In [121]:
aggs

{'channel': {'jrn_m0': <function __main__.<lambda>>,
  'jrn_m1': <function __main__.<lambda>>},
 'topics': {'jrn_m0': <function __main__.<lambda>>,
  'jrn_m1': <function __main__.<lambda>>}}

In [122]:
df_jrns.dtypes

customer_id        int64
jrn_id             int64
topics_jrn_m0     object
topics_jrn_m1     object
channel_jrn_m0    object
channel_jrn_m1    object
dtype: object

In [123]:
len("dd")

2

In [124]:
map(str.strip, str.split(',',"ss, yy")

SyntaxError: unexpected EOF while parsing (<ipython-input-124-1176a1419961>, line 1)

In [125]:
cat_cols = df_jrns.select_dtypes(include=['object']).columns
# agg_cols = filter(lambda c: re.match("agg_*",c), df_customer_journeys.columns)
# num_cols = ['int_id','jrn_id']

def tokenizer(s):    
    if s and len(s) > 0:
        tokens = str(s).split(',')        
        return map(str.strip, tokens)
    else:
        return []
        
mapper = DataFrameMapper([
        (col, CountVectorizer(min_df=2, tokenizer=tokenizer)) for col in cat_cols]         
#       + [(agg_cols+num_cols, None)]
                        ,sparse=True)

mapper.fit_transform(df_jrns).toarray()

array([[2, 1, 1, 2, 1, 1],
       [1, 1, 1, 1, 1, 1]])

In [116]:
def tokenizer(s):    
    if s and len(s) > 0:
        tokens = str(s).split(',')        
        return map(str.strip, tokens)
    else:
        return []

tokenizer(u"s, a")

['s', 'a']

In [67]:
print cat_cols
print
for (c,t) in mapper.features:
    print c, t.get_feature_names()

Index([u'topics_jrn_m0', u'topics_jrn_m1', u'channel_jrn_m0',
       u'channel_jrn_m1'],
      dtype='object')

topics_jrn_m0

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

In [441]:
grpby_columns = ['customer_id','jrn_id']

concat = lambda arr: ",".join(arr)


gr_cust_jrns = df_cust_jrns_int.groupby(by=grpby_columns, axis=0) 
df_cust_jrns = gr_cust_jrns.agg({'topics':{'m0':concat}, 'channel':{'m0':concat}})
df_cust_jrns.columns = flatten_columns(df_cust_jrns)
df_cust_jrns.reset_index()
#                                .agg(merge_dicts(count_agg(grpby_columns), \
#                                                 total_avg_agg(grpby_columns))) 

,customer_id,jrn_id,topics_m0,channel_m0
0,1,1,"t1,t2,t1,t3","web,phone,phone"
1,1,2,"t1,t3,t1","web,phone"


In [427]:
grpby_columns = ['customer_id','jrn_id']

concat = lambda arr: ";".join(arr)
totuple = lambda arr: tuple(map(lambda csv: csv.split(','), arr))


gr_cust_jrns = df_cust_jrns_int.groupby(by=grpby_columns, axis=0) 

df_cust_jrns = gr_cust_jrns.agg({'topicsM1': totuple, 'channel': totuple})
df_cust_jrns['topics'] = df_cust_jrns['topics']#.apply(list)
df_cust_jrns['channel'] = df_cust_jrns['channel']#.apply(list)
df_cust_jrns

KeyError: 'topicsM1'

In [425]:
df_cust_jrns = gr_cust_jrns.agg(aggs)
df_cust_jrns.columns = flatten_columns(df_cust_jrns)
df_cust_jrns

topics__m1   topics__m0  \
customer_id jrn_id                                              
1           1       (t1>t2, t1>t1, t2>t3, t1>t3)  t1,t2,t1,t3   
            2                     (t1>t3, t1>t1)     t1,t3,t1   

                                 channel__m1      channel__m0  
customer_id jrn_id                                             
1           1       (web>phone, phone>phone)  web,phone,phone  
            2                   (web>phone,)        web,phone

In [392]:
lists = (['t1'], ['t2', 't1'], ['t3','t4'])
# [(f,t) for f in zip(lists)]
zip(lists[:-1], lists[1:])

[(['t1'], ['t2', 't1']), (['t2', 't1'], ['t3', 't4'])]

In [393]:
import itertools
from operator import concat
z = [map(lambda l: "_".join(l), itertools.product(f_t[0],f_t[1])) for f_t in zip(lists[:-1], lists[1:])]

%timeit reduce(concat,z)
%timeit reduce(lambda x,y: x+y, z)

The slowest run took 18.70 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 255 ns per loop
The slowest run took 17.90 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 386 ns per loop


In [336]:
list(itertools.product(['t1'], ['t2', 't1', 't3']))

[('t1', 't2'), ('t1', 't1'), ('t1', 't3')]

In [197]:
c = None
for g in gr_cust_jrns:
    c = g[1]    

In [198]:
def f(row): 
    print '++++++'
    print row
    
c.apply(f, axis=1)

++++++
customer_id      1
jrn_id           2
int_id           4
channel        web
topics          t1
step_num         0
Name: 3, dtype: object
++++++
customer_id        1
jrn_id             2
int_id             5
channel        phone
topics         t3,t1
step_num           1
Name: 4, dtype: object


3    None
4    None
dtype: object

In [199]:
def f(row): 
    print '++++++'
    print type(row)    
    print row
    return row.apply(lambda s: s.split(','))
    
df_c = c[['channel', 'topics']].apply(f, axis=0)


++++++
<class 'pandas.core.series.Series'>
3      web
4    phone
Name: channel, dtype: object
++++++
<class 'pandas.core.series.Series'>
3      web
4    phone
Name: channel, dtype: object
++++++
<class 'pandas.core.series.Series'>
3       t1
4    t3,t1
Name: topics, dtype: object


In [200]:
df_c['topics'].apply(len)

3    1
4    2
Name: topics, dtype: int64

In [212]:
def f2(gr):
    print '++++++'
    print gr
    type(gr['channel'])
    return gr['topics'].T
    
zz = gr_cust_jrns.apply(f2)
zz

++++++
   customer_id  jrn_id  int_id channel topics  step_num
0            1       1       1     web     t1         0
1            1       1       2   phone  t2,t1         1
2            1       1       3   phone     t3         2
++++++
   customer_id  jrn_id  int_id channel topics  step_num
0            1       1       1     web     t1         0
1            1       1       2   phone  t2,t1         1
2            1       1       3   phone     t3         2
++++++
   customer_id  jrn_id  int_id channel topics  step_num
3            1       2       4     web     t1         0
4            1       2       5   phone  t3,t1         1


customer_id  jrn_id   
1            1       0       t1
                     1    t2,t1
                     2       t3
             2       3       t1
                     4    t3,t1
Name: topics, dtype: object

### onehot encode

In [168]:
categorical_cols = ['channelm0','topicsm1']
# agg_cols = filter(lambda c: re.match("agg_*",c), df_customer_journeys.columns)
# num_cols = ['int_id','jrn_id']

mapper = DataFrameMapper([
        (col, CountVectorizer()) for col in categorical_cols]         
#       + [(agg_cols+num_cols, None)]
                        ,sparse=True)

mapper.fit_transform(df_cust_jrns).toarray()

array([[2, 1, 2, 1, 1],
       [1, 1, 2, 0, 1]])

In [169]:
to = mapper.features[1][1]
to.get_feature_names()

[u't1', u't2', u't3']

### prepare for infer 

In [98]:
cust_jrns_int = [
    {'int_id': 1, 'jrn_id': 2, 'topics':'t2,t5', 'channel': 'web' },
    {'int_id': 2, 'jrn_id': 2, 'topics':'t5,t1', 'channel': 'phone' },
    {'int_id': 3, 'jrn_id': 3, 'topics':'t5', 'channel': 'web' }    
]
df_cust_jrns_int = pd.DataFrame(cust_jrns_int, columns=['jrn_id','int_id', 'channel','topics'])
# df_customer_journeys.set_index(['jrn_id', 'int_id'],inplace=True)
df_cust_jrns_int

,jrn_id,int_id,channel,topics
0,2,1,web,"t2,t5"
1,2,2,phone,"t5,t1"
2,3,3,web,t5


In [45]:
mapper.transform(df_gr).toarray()


array([[2, 1, 2, 1, 1]])

### explore DictVectorizer

In [268]:
df_cat = df_customer_journeys[categorical_cols]
dict_cat = df_cat.T.to_dict().values()
dict_cat

[{'channel': 'web', 'topics': 't1'},
 {'channel': 'bibush', 'topics': 't2'},
 {'channel': 'phone', 'topics': 't3'}]

In [65]:
d3 = [
    {'topics_t5':1, 'channel_web':1},
    {'topics_t2':1 ,'topics_t5':1, 'channel_phone':1 },
    {'topics_t5':1, 'channel_web':1 }    
]

In [66]:
dv = DictVectorizer(sparse=False, separator=':')
dv.fit(d3)

DictVectorizer(dtype=<type 'numpy.float64'>, separator=':', sort=True,
        sparse=False)

In [67]:
dv.transform(d3)

array([[ 0.,  1.,  0.,  1.],
       [ 1.,  0.,  1.,  1.],
       [ 0.,  1.,  0.,  1.]])

In [68]:
dv.get_feature_names()

['channel_phone', 'channel_web', 'topics_t2', 'topics_t5']

In [36]:
dict_cat2 = [
    {'topics_t5':5, 'channel_web':1},
    {'topics_t3':1 ,'topics_t5':1, 'channel_phone':1 },
    {'topics_t3':1, 'channel_bibi':1 }    
]

In [37]:
z = dv.transform(dict_cat2)
z

array([[ 0.,  1.,  0.,  5.],
       [ 1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.]])

In [72]:
cv = CountVectorizer(min_df=1)


In [78]:
corpus = [
     't1',
     't2',
     't3',
     't2',
]

In [79]:
cv.fit_transform(corpus).toarray()

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0]])

In [80]:
cv.get_feature_names()

[u't1', u't2', u't3']

In [301]:
cv.transform(['t4,t3','t3']).toarray()

array([[0, 0, 1],
       [0, 0, 1]])

In [297]:
type(cv.transform(['t2,t4']))

scipy.sparse.csr.csr_matrix

In [260]:
import scipy.sparse as sp
sp.hstack()
